# SVM Model for Tumour Classification
## A. Binary Task
### Build a classifier to identify whether there is a tumor in the MRI images.

Import necessary libraries, matplotlib, pandas, numpy, sklearn, PIL, scipy and os module

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import pandas as pd
from PIL import Image                 #Use PIL for image size reduction
import os                             #Import os module
from os import path

Read label.csv; Define original/resized image data folder path

In [2]:
csv_array = pd.read_csv('./dataset/label.csv')

dir_image = "./dataset/image/"
dir_img256 = "./dataset/image256/"

if path.exists(dir_img256) == 'false':
    os.makedirs(dir_img256)

Resize 3000 images from 512-512 to 256-256 resolution; Read resized images into "imgs" array; each image is flattened into one row and also reduced from RGB to grey scale; Modify csv_array.label to adapt to binary clasification (only "no_tumor" and "has_tumor" are valid labels)

In [3]:
rows, cols = (3000, 65536)            #rows is data set size; cols is width*length (after image size reduction. Here is 256*256)
     
imgs = np.zeros((rows,cols))          #Initialize an array for all reduced images

basewidth = 256                       #Target image width is 256; original width is 512
img = Image.open("./dataset/image/IMAGE_0000.jpg")
wpercent = float(basewidth) / float(img.size[0])       #use "256/512" as resize ratio (wpercent)
hsize = int((float(img.size[1]) * float(wpercent)))    #calculated target image height using resize ratio (wpercent)

for i in range(rows):                 #Reduce every image's size and save them in 'dir_img256' folder
    img = Image.open(dir_image + csv_array.file_name[i])
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img.save(dir_img256 + csv_array.file_name[i])

for i in range(rows):                 #Read every resized image and reduce it from RGB to grayscale format. Transform it to 1-d array. Store all images in a 2-d array, imgs. 
    img = mpimg.imread(dir_img256 + csv_array.file_name[i])[...,0]
    imgs[i] = img.flatten()
print(imgs.shape)

for i in range(rows):
    if csv_array.label[i] != "no_tumor" :
        csv_array.label[i] = "has_tumor" #only no_tumor and has_tumor are valid labels for binary clasifcation task

(3000, 65536)


Split train/test data set by 9:1

In [4]:
x_train, x_test, y_train, y_test = train_test_split(imgs, csv_array.label[0:rows], test_size=0.10, random_state=0)

Tune hyperparameter C for SVM model to get better classification accuracy

In [13]:
param_distributions = {"C": uniform(1, 99)} #Random tune C between 1 and 100

model = SVC(kernel='rbf', gamma='scale')
clf = RandomizedSearchCV(model,param_distributions, n_iter=25, random_state=0)

search = clf.fit(x_train[0:2700], y_train[0:2700]) #use training data set to tune C
search.best_estimator_

SVC(C=55.33253688880515)

Train SVM model (best_estimator_) and predict based on test data set. Print accuracy score.

In [14]:
search.best_estimator_.fit(x_train, y_train)

y_pred = search.best_estimator_.predict(x_test)
accuracy_score(y_test, y_pred)

0.9433333333333334

The rest code is for testing on new released test data set!

In [5]:
csv_array = pd.read_csv('./test-2/label.csv')

dir_image = "./test-2/image/"
dir_img256 = "./test-2/image256/"

if path.exists(dir_img256) == 'false':
    os.makedirs(dir_img256)

rows, cols = (200, 65536)            #rows is data set size; cols is width*length (after image size reduction. Here is 256*256)
     
imgs = np.zeros((rows,cols))          #Initialize an array for all reduced images

basewidth = 256                       #Target image width is 256; original width is 512
img = Image.open("./test-2/image/IMAGE_0000.jpg")
wpercent = float(basewidth) / float(img.size[0])       #use "256/512" as resize ratio (wpercent)
hsize = int((float(img.size[1]) * float(wpercent)))    #calculated target image height using resize ratio (wpercent)

for i in range(rows):                 #Reduce every image's size and save them in 'dir_img256' folder
    img = Image.open(dir_image + csv_array.file_name[i])
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img.save(dir_img256 + csv_array.file_name[i])

for i in range(rows):                 #Read every resized image and reduce it from RGB to grayscale format. Transform it to 1-d array. Store all images in a 2-d array, imgs. 
    img = mpimg.imread(dir_img256 + csv_array.file_name[i])[...,0]
    imgs[i] = img.flatten()
print(imgs.shape)

for i in range(rows):
    if csv_array.label[i] != "no_tumor" :
        csv_array.label[i] = "has_tumor" #only no_tumor and has_tumor are valid labels for binary clasifcation task

(200, 65536)


In [6]:
model = SVC(kernel='rbf', gamma='scale', C = 55.33253688880515)
model.fit(x_train, y_train)
y_pred = model.predict(imgs)
accuracy_score(csv_array.label, y_pred)

0.945